<a href="https://colab.research.google.com/github/onlyabhilash/spark-nlp-transformers/blob/main/HuggingFace_in_Spark_NLP_BERT_Sentence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20BERT%20Sentence.ipynb)

## Import BERT models for Sentence Embeddings from HuggingFace 🤗 into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.1.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for BERT from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use BERT models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.1` version and Transformers on `4.6.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q transformers==4.6.1 tensorflow==2.4.1

     |████████████████████████████████| 2.2 MB 8.4 MB/s 
     |████████████████████████████████| 394.3 MB 13 kB/s 
     |████████████████████████████████| 880 kB 50.6 MB/s 
     |████████████████████████████████| 3.3 MB 52.2 MB/s 
     |████████████████████████████████| 14.8 MB 41.2 MB/s 
     |████████████████████████████████| 132 kB 43.4 MB/s 
     |████████████████████████████████| 2.9 MB 43.4 MB/s 
     |████████████████████████████████| 3.8 MB 47.1 MB/s 
     |████████████████████████████████| 462 kB 61.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [bert-base-cased](https://huggingface.co/bert-base-cased) model from HuggingFace as an example
- In addition to `TFBertModel` we also need to save the `BertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from transformers import TFBertModel, BertTokenizer 

MODEL_NAME = 'bert-base-cased'

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME).save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

# just in case if there is no TF/Keras file provided in the model
# we can just use `from_pt` and convert PyTorch to TensorFlow
try:
  print('try downloading TF weights')
  model = TFBertModel.from_pretrained(MODEL_NAME)
except:
  print('try downloading PyTorch weights')
  model = TFBertModel.from_pretrained(MODEL_NAME, from_pt=True)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

try downloading TF weights


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

INFO:tensorflow:Assets written to: ./bert-base-cased/saved_model/1/assets


INFO:tensorflow:Assets written to: ./bert-base-cased/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {MODEL_NAME}

total 423352
-rw-r--r-- 1 root root       597 May  3 10:53 config.json
drwxr-xr-x 3 root root      4096 May  3 10:53 saved_model
-rw-r--r-- 1 root root 433503000 May  3 10:53 tf_model.h5


In [4]:
!ls -l {MODEL_NAME}/saved_model/1

total 7736
drwxr-xr-x 2 root root    4096 May  3 10:53 assets
-rw-r--r-- 1 root root 7910827 May  3 10:53 saved_model.pb
drwxr-xr-x 2 root root    4096 May  3 10:53 variables


In [5]:
!ls -l {MODEL_NAME}_tokenizer

total 220
-rw-r--r-- 1 root root    112 May  3 10:52 special_tokens_map.json
-rw-r--r-- 1 root root    521 May  3 10:52 tokenizer_config.json
-rw-r--r-- 1 root root 213450 May  3 10:52 vocab.txt


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `vocab.txt` from the tokenizer
- all we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for

In [6]:
!cp {MODEL_NAME}_tokenizer/vocab.txt {MODEL_NAME}/saved_model/1/assets

## Import and Save BERT in Spark NLP for Sentence/Document embeddings
### `BertSentenceEmbeddings` is the equivalent annotator for this task
The output of this annotator can be used in multi-class/multi-label text classifications (`ClassifierDL`, `SentimentDL`, and `MultiClassifierDL`) 


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [7]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-05-03 10:53:46--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-05-03 10:53:47--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-05-03 10:53:47--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `BertSentenceEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Unlike `BertEmbeddings` which uses `last_hidden_state` with (-1, -1, DIMENSION) shape, `BertSentenceEmbeddings` will use `pooler_output` with (-1, DIMENSION) shape for Sentence/Document embeddings. It will generate 1 vector for the entire sentence/document
- Most params can be set later when you are loading this model in `BertSentenceEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want! 
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.
- NOTE: `loadSavedModel` only accepts local paths and not distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. That is why we use `write.save` so we can use `.load()` from any file systems.


In [9]:
from sparknlp.annotator import *

sent_bert = BertSentenceEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols("sentence")\
 .setOutputCol("bert_sentence")\
 .setCaseSensitive(True)\
 .setDimension(768)\
 .setStorageRef('sent_bert_base_cased') 

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
sent_bert.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your BERT model for Sentence/Document embeddings from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [12]:
! ls -l {MODEL_NAME}_spark_nlp

total 430608
-rw-r--r-- 1 root root 440926938 May  3 10:55 bert_sentence_tensorflow
drwxr-xr-x 4 root root      4096 May  3 10:55 fields
drwxr-xr-x 2 root root      4096 May  3 10:55 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BERT model 😊 

In [13]:
sent_bert_loaded = BertSentenceEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols("sentence")\
  .setOutputCol("bert_sentence")\
  .setCaseSensitive(True)

In [14]:
sent_bert_loaded.getStorageRef()

'sent_bert_base_cased'

That's it! You can now go wild and use hundreds of BERT models from HuggingFace 🤗 in Spark NLP 🚀 
